----
<img src="../../files/lseg.svg" class="rft-examples-logo" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content layer - Pricing stream - Used as a real-time data cache
This notebook demonstrates how to retrieve level 1 streaming data (such as trades and quotes) either directly from the LSEG Data Platform or via LSEG Workspace or CodeBook. The example shows how to define a Pricing stream object, which automatically manages a streaming cache available for access at any time. Your application can then reach into this cache and pull out real-time snapshots as Pandas DataFrames by just calling a simple access method.

Using a Pricing stream object that way prevents your application from sending too many requests to the platform. This is particularly useful if your application needs to retrieve real-time snapshots at regular and short intervals.

#### Learn more

To learn more about the LSEG Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/tutorials), 
 [Documentation](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/documentation)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Data Library Q&A Forum](https://community.developers.refinitiv.com/smart-spaces/521/index.html). 
The LSEG Developer Community will be happy to help. 

----

## Some Imports to start with

In [1]:
import lseg.data as ld
from IPython.display import display

## Open the data session

The open_session() function creates and open sessions based on the information contained in the lseg-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
ld.open_session()

<lseg.data.session.Definition object at 0x10a358b20 {name='workspace'}>

## Retrieve data

### Create and open a Pricing stream object

The Pricing stream object is created for a list of instruments and fields. The fields parameter is optionnal. If you omit it, the Pricing stream will retrieve all fields available for the requested instruments

In [3]:
stream = ld.content.pricing.Definition(
    universe = ['EUR=', 'GBP=', 'JPY=', 'CAD='], 
    fields = ['BID', 'ASK']
).get_stream()

The open method tells the Pricing stream object to subscribe to the streams of the requested instruments.

In [4]:
stream.open()

<OpenState.Opened: 'Opened'>

As soon as the open method returns, the stream object is ready to be used. Its internal cache is constantly kept updated with the latest streaming information received from LSEG Workspace. All this happens behind the scene, waiting for your application to pull out data from the cache.  

### Extract snapshot data from the streaming cache
Once the stream is opened, you can use the get_snapshot method to pull out data from its internal cache. get_snapshot can be called any number of times. As these calls return the latest received values, successive calls to get_snapshot may return different values. Returned DataFrames do not change in real-time, get_snapshot must be called every time your application needs fresh values. 

In [5]:
df = stream.get_snapshot()
display(df)

,Instrument,ASK,BID
0,EUR=,1.1117,1.1116
1,GBP=,1.3171,1.317
2,JPY=,142.72,142.67
3,CAD=,1.3504,1.3503


### Get a snapshot for a subset of instruments and fields

In [6]:
df = stream.get_snapshot(
    universe = ['EUR=', 'GBP='], 
    fields = ['BID', 'ASK']
)
display(df)

,Instrument,BID,ASK
0,EUR=,1.1116,1.1117
1,GBP=,1.3168,1.3174


### Other options to get values from the streaming cache

#### Direct access to real-time fields

In [7]:
print('GBP/BID:', stream['GBP=']['BID'])
print('EUR/BID:', stream['EUR=']['BID'])

GBP/BID: 1.3168
EUR/BID: 1.1116


#### Direct acces to a streaming instrument

In [8]:
gbp = stream['GBP=']
print(gbp['BID'])

1.317


#### Iterate on fields

In [9]:
print('GBP=')
for field_name, field_value in stream['GBP=']:
    print('\t' + field_name + ': ', field_value)
    
print('JPY=')
for field_name, field_value in stream['JPY=']:
    print('\t' + field_name + ': ', field_value)

GBP=
	ASK:  1.3171
	BID:  1.317
JPY=
	ASK:  142.73
	BID:  142.68


#### Iterate on streaming instruments and fields

In [10]:
for streaming_instrument in stream:
    print(streaming_instrument.name)
    for field_name, field_value in streaming_instrument:
        print('\t' + field_name + ': ', field_value)

EUR=
	ASK:  1.1117
	BID:  1.1116
GBP=
	ASK:  1.3171
	BID:  1.317
JPY=
	ASK:  142.7
	BID:  142.69
CAD=
	ASK:  1.3504
	BID:  1.3503


### Close the stream

In [11]:
stream.close()

<OpenState.Closed: 'Closed'>

Once closed is called the Pricing stream object stops updating its internal cache. The get_snapshot function can still be called but after the close it always return the same values. 

### Invalid or un-licensed instruments
What happens if you request using an invalid RIC or an instrument you are not entitled to?
Let's request a mixture of valid and invalid RICs

In [12]:
mixed = ld.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD=', 'BADRIC'],
    fields=['BID', 'ASK']
).get_stream()

mixed.open()
mixed.get_snapshot()

,Instrument,ASK,BID
0,EUR=,1.1117,1.1116
1,GBP=,1.3171,1.317
2,JPY=,142.74,142.69
3,CAD=,1.3504,1.3503
4,BADRIC,<NA>,<NA>


You can check the Status of any instrument, so lets check the invalid one

In [13]:
display(mixed['BADRIC'].code)

'Closed'

As you will note, for an invalid instrument we get:   
{'status': <StreamState.Closed: 1>, **'code': 'NotFound'**, 'message': '** The Record could not be found'}   

However, if you are not licensed for the instrument you would see something like:  
{'status': <StreamState.Closed: 1>, **'code': 'NotEntitled'**, 'message': 'A21: DACS User Profile denied access to vendor'}   
**NOTE**: The exact wording of **message** can change over time - therefore,only use the **code** value for any programmatic decision making.

In [14]:
mixed.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [15]:
ld.close_session()